## Step 3. Variable Correlation Analysis , Standardization & Dumb Coding 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as sta

In [2]:
df = pd.read_csv('Feature_Selection.csv')

### 3.1 Variable Correlation Analysis

#### (1) Contingency Analysis  & IV-WOE Analysis with Categorical Variables

#### Contingency Analysis

In [3]:
# For categorical variables, we use Contingency Analysis to justify the relation with default condition
# The second return value is P-value, when < 0.05 there is relation
# Here we take EDUCATION as an example,actually we test LIMIT_BAL ,SEX ,EDUCATION ,MARRIAGE ,AGE & PAY_i_TEST
col_cat=['LIMIT_BAL_GROUP','SEX', 'MARRIAGE', 'EDUCATION','PAY_0_TEST','PAY_2_TEST', 'PAY_3_TEST', 'PAY_4_TEST', 'PAY_5_TEST','PAY_6_TEST']
ca_p=[sta.chi2_contingency(pd.crosstab(df[c] ,df.default))[1] for c in col_cat]
df_ca = pd.DataFrame(ca_p,index=col_cat,columns=['ca_p'])
df_ca

,ca_p
LIMIT_BAL_GROUP,1.260074e-161
SEX,3.148419e-11
MARRIAGE,8.582062e-08
EDUCATION,5.176524e-22
PAY_0_TEST,0.000000e+00
PAY_2_TEST,0.000000e+00
PAY_3_TEST,0.000000e+00
PAY_4_TEST,0.000000e+00
PAY_5_TEST,0.000000e+00
PAY_6_TEST,0.000000e+00


#### IV-WOE Analysis 

In [4]:
# calculate IV,WOE used for justify relationship
def calc(dataset,col,targe):
    '''
    col:independent variables
    targe:dependent variables
    '''
    subdata=dataset.groupby(col)[col].count()
    suby = dataset.groupby(col)[targe].sum()
    data=pd.merge(subdata,suby,how='left',left_index=True,right_index=True)
    b_total=data[targe].sum()
    total=data[col].sum()
    g_total = total - b_total
    data['bad']=data.apply(lambda x:round(x[targe]/b_total,3),axis=1)
    data['good']=data.apply(lambda x:round((x[col] - x[targe])/g_total,3),axis=1)
    data['WOE']=data.apply(lambda x:np.log(x.bad/x.good),axis=1)
    data['IV'] = data.apply(lambda x:(x.bad-x.good)* x.WOE,axis=1)
    IV=sum(data['IV'])
    WOE=sum(data['WOE'])
    return IV,WOE

In [5]:
targe='default'
dataset=df
a = [calc(dataset,c,targe) for c in col_cat]
df_IV = pd.DataFrame(a,index=col_cat,columns=['IV','WOE'])
df_IV

,IV,WOE
LIMIT_BAL_GROUP,0.156865,-2.983032
SEX,0.008381,0.033757
MARRIAGE,0.005798,0.008859
EDUCATION,0.019753,0.050561
PAY_0_TEST,0.716262,1.750558
PAY_2_TEST,0.546607,1.113845
PAY_3_TEST,0.407420,1.028289
PAY_4_TEST,0.348062,1.123734
PAY_5_TEST,0.329529,1.249417
PAY_6_TEST,0.281866,1.127477


In [6]:
df_cat = pd.merge(df_IV,df_ca,left_index=True,right_index=True)
df_cat.sort_values(by=['IV','WOE'])

,IV,WOE,ca_p
MARRIAGE,0.005798,0.008859,8.582062e-08
SEX,0.008381,0.033757,3.148419e-11
EDUCATION,0.019753,0.050561,5.176524e-22
LIMIT_BAL_GROUP,0.156865,-2.983032,1.260074e-161
PAY_6_TEST,0.281866,1.127477,0.000000e+00
PAY_5_TEST,0.329529,1.249417,0.000000e+00
PAY_4_TEST,0.348062,1.123734,0.000000e+00
PAY_3_TEST,0.407420,1.028289,0.000000e+00
PAY_2_TEST,0.546607,1.113845,0.000000e+00
PAY_0_TEST,0.716262,1.750558,0.000000e+00


#### (2) one-way ANOVA with Numerical Variables

In [7]:
# For numerical variables, we use one-way ANOVA to justify the relation with default condition
# We take BILL_AMT4 as an example, actually we test BILL_AMTi & PAY_AMTi
col_num = ['BILL_AMT'+str(i) for i in np.arange(1,7)] + ['PAY_AMT'+str(i) for i in np.arange(1,7)]
p_=[]
for c in col_num:
    d1=df[df['default']==0][c]
    d2=df[df['default']==1][c]
    args=[d1,d2]
    # Before one-way ANOVA, we take homogeneity test of variance first.
    w,p=sta.levene(*args)
    f,p=sta.f_oneway(*args)
    p_.append(p)
df_num = pd.DataFrame(p_,columns=['p_value'],index=col_num)
df_num.sort_values(by='p_value')

,p_value
PAY_AMT1,1.771477e-39
PAY_AMT2,4.625058e-25
PAY_AMT4,7.646590e-23
PAY_AMT3,1.156218e-22
PAY_AMT5,1.510917e-21
PAY_AMT6,1.519015e-20
BILL_AMT1,1.125713e-03
BILL_AMT2,1.977091e-02
BILL_AMT3,2.030948e-02
BILL_AMT4,1.032026e-01


Under the significance level of 0.05, we kick out BILL_AMT4, BILL_AMT5, and BILL_AMT6 three continuous variables.

### 3.2 Data Standardization

In [8]:
# In order to eliminate the influence of dimension between variables and on model results, 
# continuous variables  BILL_AMT1-6 & PAY_AMT1-6 are standardized.
def standard_scalar(data, columns):
    for column in columns:
        std = np.std(data[column])
        if std == 0:
            raise ValueError("invalid parameter: standard deviation is 0 in the '" + column + "' column.")
        else:
            mean = sum(data[column]) / len(data[column])
            data[column] = ( data[column] - mean ) / std
    return data
standard_scalar(df,df.columns[12:15])
standard_scalar(df,df.columns[18:24])
pd.set_option('display.max_columns', None)
standard_scalar(df,df.columns[1:2]).head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,AGE_GROUP,LIMIT_BAL_GROUP,PAY_0_TEST,PAY_2_TEST,PAY_3_TEST,PAY_4_TEST,PAY_5_TEST,PAY_6_TEST,Limit_Usage,Pay_Amt_std
0,1,-1.138505,Female,University,Married,24,2,2,-1,-1,-2,-2,-0.641227,-0.646464,-0.667310,0,0,0,-0.355537,-0.234479,-0.307629,-0.313593,-0.316060,-0.293887,1,20s,0-50k,Delay,Delay,Paid,Paid,Paid,Paid,0.195650,2.449490
1,2,-0.369733,Female,University,Single,26,-1,2,0,0,0,2,-0.657978,-0.665835,-0.638545,3272,3455,3261,-0.355537,-0.220403,-0.248100,-0.248590,-0.316060,-0.180938,1,20s,100-150k,Paid,Delay,Paid,Paid,Paid,Delay,0.022350,0.903327
2,3,-0.600364,Female,University,Single,34,0,0,0,0,0,0,-0.296607,-0.492770,-0.481558,14331,14948,15549,-0.259775,-0.197774,-0.248100,-0.248590,-0.250175,-0.011514,0,30s,50-100k,Paid,Paid,Paid,Paid,Paid,Paid,0.324878,0.854864
3,4,-0.907873,Female,University,Married,37,0,0,0,0,0,0,-0.055062,-0.011556,0.034160,28314,28959,29547,-0.229369,-0.174285,-0.236195,-0.242089,-0.245629,-0.237412,0,30s,0-50k,Paid,Paid,Paid,Paid,Paid,Paid,0.939800,0.341959
4,5,-0.907873,Male,University,Married,57,-1,0,-1,0,0,0,-0.577218,-0.610337,-0.160050,20940,19146,19131,-0.229369,1.394468,0.287654,0.271442,-0.270665,-0.255541,0,50s,0-50k,Paid,Paid,Paid,Paid,Paid,Paid,0.172340,1.400826


In [9]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,AGE_GROUP,LIMIT_BAL_GROUP,PAY_0_TEST,PAY_2_TEST,PAY_3_TEST,PAY_4_TEST,PAY_5_TEST,PAY_6_TEST,Limit_Usage,Pay_Amt_std
0,1,-1.138505,Female,University,Married,24,2,2,-1,-1,-2,-2,-0.641227,-0.646464,-0.667310,0,0,0,-0.355537,-0.234479,-0.307629,-0.313593,-0.316060,-0.293887,1,20s,0-50k,Delay,Delay,Paid,Paid,Paid,Paid,0.195650,2.449490
1,2,-0.369733,Female,University,Single,26,-1,2,0,0,0,2,-0.657978,-0.665835,-0.638545,3272,3455,3261,-0.355537,-0.220403,-0.248100,-0.248590,-0.316060,-0.180938,1,20s,100-150k,Paid,Delay,Paid,Paid,Paid,Delay,0.022350,0.903327
2,3,-0.600364,Female,University,Single,34,0,0,0,0,0,0,-0.296607,-0.492770,-0.481558,14331,14948,15549,-0.259775,-0.197774,-0.248100,-0.248590,-0.250175,-0.011514,0,30s,50-100k,Paid,Paid,Paid,Paid,Paid,Paid,0.324878,0.854864
3,4,-0.907873,Female,University,Married,37,0,0,0,0,0,0,-0.055062,-0.011556,0.034160,28314,28959,29547,-0.229369,-0.174285,-0.236195,-0.242089,-0.245629,-0.237412,0,30s,0-50k,Paid,Paid,Paid,Paid,Paid,Paid,0.939800,0.341959
4,5,-0.907873,Male,University,Married,57,-1,0,-1,0,0,0,-0.577218,-0.610337,-0.160050,20940,19146,19131,-0.229369,1.394468,0.287654,0.271442,-0.270665,-0.255541,0,50s,0-50k,Paid,Paid,Paid,Paid,Paid,Paid,0.172340,1.400826


In [10]:
# Store the intermediate data.
df.to_csv('data_standardized.csv', index = False)

### 3.3 Dumb Coding

In [11]:
# After significant correlation test + standardization + discretization,we get dummy coding of categorical data.
modeldata = df[['LIMIT_BAL_GROUP', 'SEX', 'MARRIAGE', 'EDUCATION','PAY_0_TEST','PAY_2_TEST', 'PAY_3_TEST', 'PAY_4_TEST', 'PAY_5_TEST','PAY_6_TEST']]
X = pd.get_dummies(modeldata)
X[list(df.columns[12:15])]=df[df.columns[12:15]]
X[list(df.columns[18:24])]=df[df.columns[18:24]]
X['Limit_Usage']=df['Limit_Usage']
X['Pay_Amt_std']=df['Pay_Amt_std']
y = df.default

In [12]:
# Store the intermediate data.
X['default'] = y
X.to_csv('data_standardized_dumb.csv', index = False)